# Risk Parity

### Background



### Summary


### Data Dictionary




#### Gold
Gold has long been considered a "safe-haven" asset, often retaining value or even appreciating during times of economic uncertainty, inflation, or market volatility. Its value doesn't typically move in direct correlation with stock or bond markets, making it a useful tool for diversification and risk management. In a risk parity portfolio, gold can serve to balance out the risks associated with more volatile assets, providing stability and preserving capital during downturns.

#### Bitcoin
Bitcoin, on the other hand, represents a more volatile and relatively newer asset class often categorized as "digital gold" or a speculative investment. However, its distinct market behavior, potential for high returns, and increasing acceptance as a legitimate investment option allow it to serve a unique role in a risk parity strategy. Bitcoin's low correlation with traditional asset classes like stocks and bonds can contribute to diversification. Moreover, for investors willing to accept higher volatility for the chance of higher returns, Bitcoin can offer a counterbalance to the more stable portions of the portfolio, like gold and bonds.

#### Combining Gold and Bitcoin in Risk Parity
Incorporating both gold and Bitcoin into a risk parity portfolio leverages their distinct characteristics — gold's stability and Bitcoin's growth potential. The idea is not to predict which asset will perform better but to allocate assets in a way that each contributes equally to the portfolio's risk profile. This can mean adjusting the allocation to gold to counterbalance the higher volatility of Bitcoin, thereby achieving a more stable and diversified investment portfolio.

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import datetime
from datetime import timedelta
from dateutil import rrule
import time

import plotly.graph_objects as go

### Data Uploading

In [51]:
SPY = pd.read_csv('Historical_Data_Prices_Cleaned/SPY.csv')
GLD = pd.read_csv('Historical_Data_Prices_Cleaned/GLD.csv')
BTC_USD = pd.read_csv('Historical_Data_Prices_Cleaned/BTC-USD.csv')

In [52]:
SPY.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7841 entries, 0 to 7840
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           7841 non-null   object 
 1   Open           7841 non-null   float64
 2   High           7841 non-null   float64
 3   Low            7841 non-null   float64
 4   Close          7841 non-null   float64
 5   Volume         7841 non-null   int64  
 6   Dividends      7841 non-null   float64
 7   Stock Splits   7841 non-null   float64
 8   Capital Gains  7841 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 551.4+ KB


In [53]:
SPY.info

<bound method DataFrame.info of             Date        Open        High         Low       Close     Volume  \
0     1993-01-29   24.781353   24.781353   24.658063   24.763741    1003200   
1     1993-02-01   24.781345   24.939861   24.781345   24.939861     480500   
2     1993-02-02   24.922258   25.010323   24.869420   24.992710     201300   
3     1993-02-03   25.027931   25.274512   25.010318   25.256899     529400   
4     1993-02-04   25.344957   25.415408   25.063151   25.362570     531500   
...          ...         ...         ...         ...         ...        ...   
7836  2024-03-14  515.368682  515.528221  510.234671  513.354980  110171800   
7837  2024-03-15  510.209991  511.700012  508.119995  509.829987  107585800   
7838  2024-03-18  514.000000  515.479980  512.440002  512.859985   88893300   
7839  2024-03-19  512.150024  516.000000  511.119995  515.710022   60755300   
7840  2024-03-20  515.770020  520.619995  515.080017  520.479980   69594600   

      Dividends  St

### Merging Datframes

In [54]:

# Ensure 'Date' columns are in datetime format for all dataframes
SPY['Date'] = pd.to_datetime(SPY['Date'])
BTC_USD['Date'] = pd.to_datetime(BTC_USD['Date'])
GLD['Date'] = pd.to_datetime(GLD['Date'])

# Merging the dataframes based on the 'Date' column
# Using an outer join to ensure all dates are included even if one asset does not have a corresponding entry
merged_df = SPY[['Date', 'Close']].rename(columns={'Close': 'SPY'}).merge(
    BTC_USD[['Date', 'Close']].rename(columns={'Close': 'BTC_USD'}), on='Date', how='outer').merge(
    GLD[['Date', 'Close']].rename(columns={'Close': 'GLD'}), on='Date', how='outer')

# Sorting by date to ensure the data is in chronological order
merged_df.sort_values(by='Date', inplace=True)

# Showing the first few rows to verify the merge
merged_df.head(), merged_df.tail()


(        Date        SPY  BTC_USD  GLD
 0 1993-01-29  24.763741      NaN  NaN
 1 1993-02-01  24.939861      NaN  NaN
 2 1993-02-02  24.992710      NaN  NaN
 3 1993-02-03  25.256899      NaN  NaN
 4 1993-02-04  25.362570      NaN  NaN,
            Date         SPY       BTC_USD         GLD
 8919 2024-03-16         NaN  65315.117188         NaN
 8920 2024-03-17         NaN  68390.625000         NaN
 7838 2024-03-18  512.859985  67548.593750  200.029999
 7839 2024-03-19  515.710022  61912.773438  199.800003
 7840 2024-03-20  520.479980  67913.671875  202.179993)

In [55]:
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

In [56]:
merged_df.tail(10)

,Date,SPY,BTC_USD,GLD
7833,2024-03-11,509.696350,72123.906250,202.000000
7834,2024-03-12,515.179321,71481.289062,199.789993
7835,2024-03-13,514.371826,73083.500000,201.190002
7836,2024-03-14,513.354980,71396.593750,200.350006
7837,2024-03-15,509.829987,69403.773438,199.710007
8919,2024-03-16,NaN,65315.117188,NaN
8920,2024-03-17,NaN,68390.625000,NaN
7838,2024-03-18,512.859985,67548.593750,200.029999
7839,2024-03-19,515.710022,61912.773438,199.800003
7840,2024-03-20,520.479980,67913.671875,202.179993


#### Drop the Weekend Dates

In domestic US equity markets, trading only occurs between weekdays from 9:30AM-4PM EST. So two of the three assets in this risk parity portfolio, SPY and GLD, both being ETFs they only have closing prices on weekdays. [Spot Bitcoin](https://finance.yahoo.com/quote/BTC-USD/profile) as used by Yahoo Finance prints closing prices all days of the week, and because of that we must remove the closing price of Spot Bitcoin, or BTC-USD on Saturdays and Sundays. After doing so, there will consistency amongst the data, and we do not need to impute values for SPY and GLD over the weekend throughout the entire dataset.

Doing this is important because it adds integrity to the dataset and gives a more accurate representation of the financial metrics, rations, data analysis and findings we derive throughout the rest of this project.

In [62]:
# Filter out weekends where the day of the week is 5 (Saturday) or 6 (Sunday)
merged_df = merged_df[merged_df['Date'].dt.weekday < 5]
merged_df.tail(10)

,Date,SPY,BTC_USD,GLD
7831,2024-03-07,513.215393,66925.484375,199.940002
7832,2024-03-08,510.134979,68300.093750,201.630005
7833,2024-03-11,509.696350,72123.906250,202.000000
7834,2024-03-12,515.179321,71481.289062,199.789993
7835,2024-03-13,514.371826,73083.500000,201.190002
7836,2024-03-14,513.354980,71396.593750,200.350006
7837,2024-03-15,509.829987,69403.773438,199.710007
7838,2024-03-18,512.859985,67548.593750,200.029999
7839,2024-03-19,515.710022,61912.773438,199.800003
7840,2024-03-20,520.479980,67913.671875,202.179993


#### Exploratory Data Analysis

In [59]:
merged_df.corr()

,Date,SPY,BTC_USD,GLD
Date,1.000000,0.868521,0.779512,0.813661
SPY,0.868521,1.000000,0.897274,0.701543
BTC_USD,0.779512,0.897274,1.000000,0.794016
GLD,0.813661,0.701543,0.794016,1.000000


### Index Calculation